In [18]:
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch

train = pd.read_csv("train.csv")
dev = pd.read_csv("dev.csv")
test = pd.read_csv("test.csv")
train.head()

,Id,text,Category
0,f6d77ca7b407bec6f00d3b94c5b11907a8781b85661e0b...,什麼 才 是 想要 的,0
1,a519e92b4570f3ef68482bb034ab6f26903aa913548aa4...,應該 是 學姊 吧 ！,1
2,b2058b0cbbac5d6c80b28219d4b685f893bb2220a614fd...,我 想要 瘦 啊 希望 能 趁 這 次 暑假 來 個 減肥 大計 祈禱 我 能 持之...,1
3,3917286757d7a0ca1b3a4d89a887a40faa8b9523749d2c...,順利 到 12月 底 完成 ！ ！ ！ ！ ！,1
4,f047ccbe9c84987ea2e99b63727e1a2597bf80815b4c20...,在 感情 裡 付出 過 的 一切 就是 屬於 兩 人 永遠 的 記憶 別人 都 無法...,0


In [33]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

BOW_vectorizer = CountVectorizer() 
BOW_500 = CountVectorizer(max_features=50, tokenizer=nltk.word_tokenize) 
BOW_500.fit(train['text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x7f8339b82cb0>,
        vocabulary=None)

In [34]:
X_train = BOW_500.transform(train['text'])
y_train = train['Category']

X_test = BOW_500.transform(test['text'])
y_test = test['Category']

print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (80000, 50)
y_train.shape:  (80000,)
X_test.shape:  (10000, 50)
y_test.shape:  (10000,)


In [35]:
X_train[0][0]

<1x50 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [32]:
import torch.utils.data as data
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)


train_loader = data.DataLoader(train, batch_size = 10, shuffle = True)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [14]:
import torch.utils.data as data
train_dataset = data.DataLoader(dataset = X_train,batch_size=64,shuffle=True)
test_dataset = data.DataLoader(dataset = X_test , batch_size=64)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [12]:

from tqdm import tqdm_notebook as tqdm

class network(nn.Module):
    def __init__(self):
        super(network,self).__init__()
        self.L1 = nn.Linear(500,256)
        self.L2 = nn.Linear(256,128)
        self.L3 = nn.Linear(128,32)
        self.output = nn.Linear(32,2)
    def forward(self , x):
        x = F.relu(self.L1(x))
        x = F.relu(self.L2(x))
        x = F.relu(self.L3(x))
        x = self.output(x)
        return x

In [13]:
net = network()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=net.parameters())
loss_history = []
batch_size = 32
for e in tqdm(range(20)):
    epoch_loss_sum = 0
    for x , y in tqdm(zip(X_train, y_train)):
        x = x.view(batch_size,-1)
        net_out = net(x)
        loss = loss_fn(net_out , y)
        epoch_loss_sum += float(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_history.append(epoch_loss_sum)

AttributeError: view not found

In [8]:
aaa = [[1,2,3], [4,5,6], [7,8,9]]
bbb = [1,2,3]
for x, y in zip(aaa, bbb):
    print(x,y)

[1, 2, 3] 1
[4, 5, 6] 2
[7, 8, 9] 3
